# Imports and Utils

In [1]:
%env CUDA_VISIBLE_DEVICES=

env: CUDA_VISIBLE_DEVICES=


In [1]:
import jax
import jax.numpy as jnp
import jax.random as jr
import numpy as np
from typing import NamedTuple
import matplotlib.pyplot as plt
import equinox as eqx
from datetime import datetime

from flowlenia.flowlenia_params import FlowLeniaParams as FLP, State, Config, beam_mutation
from flowlenia.utils import conn_from_matrix
from flowlenia.simutils import Simulator

try:
	import _pickle as pickle 
except:
	import pickle
import gzip

In [2]:
def get_img(state):
    A, P = state.A, state.P
    return P[...,:3] * A.sum(-1, keepdims=True)

import os
os.environ['FFMPEG_BINARY'] = 'ffmpeg'
import moviepy.editor as mvp
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from IPython.display import HTML, display, clear_output

class VideoWriter:
    def __init__(self, filename, fps=30.0, **kw):
        self.writer = None
        self.params = dict(filename=filename, fps=fps, **kw)
    
    def add(self, img):
        img = np.asarray(img)
        if self.writer is None:
            h, w = img.shape[:2]
            self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
        if img.dtype in [np.float32, np.float64]:
              img = np.uint8(img.clip(0, 1)*255)
        if len(img.shape) == 2:
            img = np.repeat(img[..., None], 3, -1)
        self.writer.write_frame(img)
    
    def close(self):
        if self.writer:
            self.writer.close()
    
    def __enter__(self):
        return self
    
    def __exit__(self, *kw):
        self.close()
    
    def show(self, **kw):
        self.close()
        fn = self.params['filename']
        display(mvp.ipython_display(fn, **kw))

def display_states(states, filename, **kws): 
    with VideoWriter(filename, **kws) as vid:
        for i in range(len(states)):
            im = get_img(states[i])
            vid.add(im)
        vid.show(width=512, height=512)
        vid.close()

In [3]:
SAVE_DIR = "../evoflow_saves"

# MultiSpecies

In [4]:
from models.simple import SimpleFLP as SFLP, Config as SConfig
from flowlenia.flowlenia_params import Config

def transform_fn(data):
    """transform data before saving (runs XLA side)"""
    return {"t": data["t"], "s": data["s"]}
def host_transform_fn(data):
    """transform data befor saving (runs on host side)"""
    s = data["s"]
    A, P = s.A, s.P
    fP = P.reshape((-1, P.shape[-1]))
    uP, iP, cP = np.unique(fP, axis=0, return_counts=True, return_inverse=True)
    return {"uP": uP, "cP": cP, "iP": iP, "A": A.sum(-1)}

def make_run(cfg, key, save_path, T=500_000, save_freq=100):
    """make a single run"""
    mdl_key, sim_key = jr.split(key)
    # --- Instantiate model ---
    flp = SFLP(cfg, key=mdl_key)
    # --- Setup simulator ---
    sim = Simulator(flp, save_pth=save_path, zip_files=True, save_freq=save_freq, transform_fn=transform_fn, host_transform_fn=host_transform_fn)
    print("Saving at: ", save_path)
    # --- Simulate ---
    s = sim.simulate(T, sim_key)

def make_runs(cfg, T, seed, n_seeds):
    """make multiple runs with n_seeds different random seeds"""
    date = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    save_path = f"{SAVE_DIR}/multispecies/{date}_{seed}"
    os.makedirs(save_path)
    with open(f"{save_path}/config.pickle", "wb") as file:
        pickle.dump(cfg, file)
    key = jr.key(seed)
    keys = jr.split(key, n_seeds)
    for i, key in enumerate(keys):
        seed_save_path = save_path+"/"+str(i)
        os.makedirs(seed_save_path)
        make_run(cfg, key, seed_save_path, T=T)

In [5]:
# Set the channel to channel connection matrix (M[i,j] = number of kerens from channel i to channel j)
M = np.array([[5, 5, 5],
              [5, 5, 5],
              [5, 5, 5]], dtype = int)
C = M.shape[0] # number of channels
k = jnp.sum(M) # total number of kernels
c0, c1 = conn_from_matrix(M) # out-channels and in-channels
flp_cfg = Config(
    X=512, #width of grid
    Y=512, #height of grid
    C=C,   #number of channels
    k=k,   #number of kernels
    c0=c0, #out-channels
    c1=c1, # in-channels
    mix_rule="stoch", #parameter mixing rule
    crossover_rate=0., #crossover rate
)
cfg = SConfig(flp_cfg=flp_cfg,
             mutation_rate=0.001)

In [ ]:
# make a single run

save_path = f"{SAVE_DIR}/multispecies_test" #where  should run data be saved
make_run(cfg, jax.random.key(0), save_path, T=100_000) #start run

path ../evoflow_saves/multispecies_test already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/multispecies_test


  0%|          | 310/100000 [16:27<89:49:32,  3.24s/it]

In [10]:
# Make 5 runs with different seeds

seed = 10
make_runs(cfg, seed, 5)

path ../evoflow_saves/multispecies/06_04_2024_15_18_32_10/0 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/multispecies/06_04_2024_15_18_32_10/0


100%|█████████████████████████████████████████████▉| 499710/500000 [2:59:57<00:06, 44.65it/s]

path ../evoflow_saves/multispecies/06_04_2024_15_18_32_10/1 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/multispecies/06_04_2024_15_18_32_10/1


100%|█████████████████████████████████████████████▉| 499810/500000 [3:00:00<00:04, 44.34it/s]WARNING:py.warnings:/home/erwan/miniconda3/envs/jax/lib/python3.11/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(

 72%|█████████████████████████████████             | 359510/500000 [2:08:08<51:31, 45.45it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# Dissipative

In [5]:
from models.dissipative import DissipativeFLP as DFLP, Config as DConfig
from flowlenia.flowlenia_params import Config

In [6]:
def transform_fn(data):
    return {"t": data["t"], "s": data["s"]}
def host_transform_fn(data):
    s = data["s"]
    A, P = s.A, s.P
    fP = P.reshape((-1, P.shape[-1]))
    uP, iP, cP = np.unique(fP, axis=0, return_counts=True, return_inverse=True)
    return {"uP": uP, "cP": cP, "iP": iP, "A": A.sum(-1)}

def make_dissipative_run(cfg, key, save_path):
    mdl_key, sim_key = jr.split(key)
    # --- Instantiate model ---
    flp = DFLP(cfg, key=mdl_key)
    # --- Setup simulator ---
    sim = Simulator(flp, save_pth=save_path, zip_files=True, save_freq=100, transform_fn=transform_fn, host_transform_fn=host_transform_fn)
    print("Saving at: ", save_path)
    # --- Simulate ---
    T = 500_000
    s = sim.simulate(T, sim_key)

def make_dissipative_runs(cfg, seed, n_seeds):
    date = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    save_path = f"{SAVE_DIR}/dissipative/{date}_{seed}"
    os.makedirs(save_path)
    with open(f"{save_path}/config.pickle", "wb") as file:
        pickle.dump(cfg, file)
    key = jr.key(seed)
    keys = jr.split(key, n_seeds)
    for i, key in enumerate(keys):
        seed_save_path = save_path+"/"+str(i)
        os.makedirs(seed_save_path)
        make_dissipative_run(cfg, key, seed_save_path)

In [9]:
C = 3
M = np.full((C,C), 5, dtype = int)
k = jnp.sum(M)
c0, c1 = conn_from_matrix(M)

flp_cfg = Config(
    X=512,
    Y=512,
    C=C,
    k=k,
    c0=c0,
    c1=c1,
    mix_rule="stoch"
)

cfg = DConfig(
    flp_cfg = flp_cfg,
    n_init_species=64,
    mutation_rate=0.0001,
    beam_prob=.01,
)

In [10]:
seed = 1
make_dissipative_runs(cfg, seed, 5)

path ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/0 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 499710/500000 [2:44:28<00:05, 50.18it/s]

path ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/1 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 499710/500000 [2:47:21<00:05, 49.59it/s]

path ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/2 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 499710/500000 [2:46:24<00:05, 50.58it/s]

path ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/3 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/3


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 499710/500000 [2:45:52<00:05, 49.62it/s]

path ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/4 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/dissipative/02_29_2024_08_48_04_1/4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 499710/500000 [2:41:05<00:05, 51.96it/s]

# Food

In [13]:
from models.food import FoodFLP as FFLP, Config as FConfig

def transform_fn(data):
    return {"t": data["t"], "s": data["s"]}
def host_transform_fn(data):
    s = data["s"]
    A, P = s.A, s.P
    fP = P.reshape((-1, P.shape[-1]))
    uP, iP, cP = jnp.unique(fP, axis=0, return_counts=True, return_inverse=True)
    return {"uP": uP, "cP": cP, "iP": iP, "A": A.sum(-1)}

def make_food_run(cfg, key, save_path):
    mdl_key, sim_key = jr.split(key)
    # --- Instantiate model ---
    flp = FFLP(cfg, key=mdl_key)
    # --- Setup simulator ---
    sim = Simulator(flp, save_pth=save_path, zip_files=True, save_freq=100, transform_fn=transform_fn, host_transform_fn=host_transform_fn)
    print("Saving at: ", save_path)
    # --- Simulate ---
    T = 500_000
    s = sim.simulate(T, sim_key)

def make_food_runs(cfg, seed, n_seeds):
    date = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    save_path = f"{SAVE_DIR}/food/{date}_{seed}"
    os.makedirs(save_path)
    with open(f"{save_path}/config.pickle", "wb") as file:
        pickle.dump(cfg, file)
    key = jr.key(seed)
    keys = jr.split(key, n_seeds)
    for i, key in enumerate(keys):
        seed_save_path = save_path+"/"+str(i)
        os.makedirs(seed_save_path)
        make_food_run(cfg, key, seed_save_path)

In [16]:
C = 4
M = np.full((C,C), 4, dtype = int)
M[:, 0] = 0
k = jnp.sum(M)
c0, c1 = conn_from_matrix(M)

flp_cfg = Config(
    X=512,
    Y=512,
    C=C,
    k=k,
    c0=c0,
    c1=c1,
    mix_rule="stoch",
)

cfg = FConfig(
    flp_cfg = flp_cfg,
    n_init_species=64,
    mutation_rate=0.001,
    food_to_matter_ratio=1.,
    decay_rate=.0001,
    food_birth_rate=0.1
)

In [ ]:
seed = 1
make_food_runs(cfg, seed, 5)

path ../evoflow_saves/food/02_26_2024_10_36_15_1/0 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/food/02_26_2024_10_36_15_1/0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [1:50:00<00:00, 75.75it/s]
  warnings.warn(



path ../evoflow_saves/food/02_26_2024_10_36_15_1/1 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/food/02_26_2024_10_36_15_1/1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [1:49:38<00:00, 76.01it/s]


path ../evoflow_saves/food/02_26_2024_10_36_15_1/2 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/food/02_26_2024_10_36_15_1/2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [1:49:15<00:00, 76.27it/s]


path ../evoflow_saves/food/02_26_2024_10_36_15_1/3 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/food/02_26_2024_10_36_15_1/3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [1:49:35<00:00, 76.04it/s]


path ../evoflow_saves/food/02_26_2024_10_36_15_1/4 already exist, simulating can overwrite content
Saving at:  ../evoflow_saves/food/02_26_2024_10_36_15_1/4


 26%|█████████████████████████████████████▉                                                                                                              | 128305/500000 [28:02<1:15:42, 81.83it/s]